In [1]:
# module import
import pandas as pd
import numpy as np

In [2]:
combat = pd.read_csv('train_payment.csv')

In [3]:
combat.tail()

,day,acc_id,amount_spent
66427,1,13809,1.056123
66428,2,13809,0.697041
66429,17,13809,0.117347
66430,15,17325,0.011735
66431,5,27258,0.035204


In [4]:
# make new column
combat["wk"] = 0

In [5]:
# month classification by week
def wk_classification(df):
    df.loc[df['day'] <= 7, ['wk']] = 1
    df.loc[(df['day'] > 7) & (df['day'] <= 14), ['wk']] = 2
    df.loc[(df['day'] > 14) & (df['day'] <= 21), ['wk']] = 3
    df.loc[df['day'] >  21, ['wk']] = 4
    return

wk_classification(combat)

In [6]:
# set index for groupby
combat.set_index(combat["acc_id"], inplace = True)

In [7]:
combat.head()

,day,acc_id,amount_spent,wk
acc_id,,,,
27835,18,27835,0.826123,3
27835,23,27835,0.011735,4
27835,27,27835,0.035204,4
12351,17,12351,0.528062,3
125437,11,125437,0.633674,2


In [9]:
combat.columns[:2]

Index(['day', 'acc_id'], dtype='object')

In [10]:
# delete column to same index
combat = combat.drop(combat.columns[:2], axis=1) 

In [11]:
# confirm indexing
combat.sort_index(ascending = True).head(10)

,amount_spent,wk
acc_id,,
8,0.348521,4
8,1.056123,1
20,0.826123,3
20,0.035204,2
20,0.035204,2
21,0.348521,4
21,2.346940,3
38,5.280616,4
38,0.586735,2


In [12]:
# groupby 'acc_id', "wk" and sum of other columns
t_group = combat.groupby(['acc_id', "wk"], sort = ['acc_id'])[combat.columns[:]].agg('sum')
t_group.head()

amount_spent  wk
acc_id wk                  
8      1       1.056123   1
       4       0.348521   4
20     2       0.070408   4
       3       0.826123   3
21     3       2.346940   3

In [13]:
act_wk1 = combat[combat["wk"]==1]
act_wk1.sort_index(ascending = True).head()

,amount_spent,wk
acc_id,,
8,1.056123,1
38,0.442398,1
43,3.696431,1
53,0.023469,1
53,0.477602,1


In [14]:
combat.columns

Index(['amount_spent', 'wk'], dtype='object')

In [15]:
act_wk1 = combat[combat["wk"]==1].groupby("acc_id").sum()
act_wk1.rename(columns={ 'amount_spent':'amount_spent_1'},inplace=True)
act_wk2 = combat[combat["wk"]==2].groupby("acc_id").sum()
act_wk2.rename(columns={ 'amount_spent':'amount_spent_2'},inplace=True)
act_wk3 = combat[combat["wk"]==3].groupby("acc_id").sum()
act_wk3.rename(columns={ 'amount_spent':'amount_spent_3'},inplace=True)
act_wk4 = combat[combat["wk"]==4].groupby("acc_id").sum()
act_wk4.rename(columns={ 'amount_spent':'amount_spent_4'},inplace=True)
print(act_wk1)

        amount_spent_1  wk
acc_id                    
8             1.056123   1
38            0.442398   1
43            3.696431   1
53            0.501072   2
62            1.584185   2
67            3.542706   2
69            0.348521   1
129           0.176021   1
133           0.398980   4
138           0.035204   1
154           2.170920   3
173           1.466838   1
203           4.224493   2
204           1.173470   2
205           1.056123   1
226           0.234694   1
247           1.470358   1
251           0.116174   1
253           0.348521   1
257           0.035204   1
266           0.243964   1
273           1.056123   1
281           0.348521   1
283           0.536276   2
287           5.875565   3
295           0.348521   1
309           0.442398   1
310           1.408164   2
335           5.351024   4
337           0.348521   1
...                ...  ..
130230        0.348521   1
130246        0.528062   1
130250        1.056123   1
130255        0.234694   1
1

In [16]:
act_wk1.reset_index(inplace=True)
act_wk2.reset_index(inplace=True)
act_wk3.reset_index(inplace=True)
act_wk4.reset_index(inplace=True)

In [17]:
del act_wk1["wk"]
del act_wk2["wk"]
del act_wk3["wk"]
del act_wk4["wk"]

In [18]:
print(act_wk1.shape)
print(act_wk2.shape)
print(act_wk3.shape)
print(act_wk4.shape)

(11768, 2)
(9293, 2)
(11772, 2)
(12357, 2)


In [19]:
act_mer1 = pd.merge(act_wk1, act_wk2, how='outer', on = "acc_id")
act_mer1.head(10)
act_mer1.shape

(16252, 3)

In [20]:
act_mer2 = pd.merge(act_wk3, act_wk4, how='outer', on = "acc_id")
act_mer2.head(10)
act_mer2.shape

(17984, 3)

In [21]:
act_mer = pd.merge(act_mer1, act_mer2, how = "outer", on = "acc_id").fillna(0)
act_mer

,acc_id,amount_spent_1,amount_spent_2,amount_spent_3,amount_spent_4
0,8,1.056123,0.000000,0.000000,0.348521
1,38,0.442398,0.586735,0.000000,5.280616
2,43,3.696431,0.000000,0.000000,0.000000
3,53,0.501072,0.000000,0.000000,0.000000
4,62,1.584185,0.000000,2.335206,0.371990
5,67,3.542706,0.000000,0.000000,1.396429
6,69,0.348521,0.000000,0.000000,0.000000
7,129,0.176021,0.826123,0.880103,0.000000
8,133,0.398980,0.246429,1.008011,0.327398
9,138,0.035204,0.000000,3.519237,1.173470


In [22]:
activity_df = pd.read_csv('payment.csv')

In [23]:
print(activity_df.shape)
activity_df.head()

(23726, 2)


,acc_id,sum_spent
0,8,1.404644
1,20,0.896531
2,21,2.695461
3,38,6.309749
4,41,0.035204


In [26]:
ds = pd.merge(activity_df,act_mer,how = "outer")
print(ds.shape)
ds.tail()

(23726, 6)


,acc_id,sum_spent,amount_spent_1,amount_spent_2,amount_spent_3,amount_spent_4
23721,130447,3.215308,1.056123,1.091327,1.056123,0.011735
23722,130449,0.348521,0.000000,0.000000,0.348521,0.000000
23723,130469,0.360255,0.000000,0.000000,0.360255,0.000000
23724,130470,1.392909,0.035204,1.228623,0.129082,0.000000
23725,130473,2.955971,1.219235,0.293368,1.372960,0.070408


In [25]:
ds.to_csv('08-22train_payment.csv',index=False)